_There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. As a consultant working for a real estate start-up, you have collected Airbnb listing data from various sources to investigate the short-term rental market in New York. You'll analyze this data to provide insights on private rooms to the real estate company._

Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [123]:
import pandas as pd
import numpy as np

# load in all the data-sets as individual dfs (initially)
price_df = pd.read_csv('data/airbnb_price.csv')
roomtype_df = pd.read_excel('data/airbnb_room_type.xlsx')
lastreview_df = pd.read_csv('data/airbnb_last_review.tsv', sep='\t')

Though implied from the project instruction, I want to check that the 'listing_ID' is essentially an intersection.

In [138]:
# check the listing_id is an intersection between the three datasets
if all('listing_id' in df.columns for df in [price_df, roomtype_df, lastreview_df]):
    id_sets = [set(df['listing_id']) for df in [price_df, roomtype_df, lastreview_df]]
    shared_ids = set.intersection(*id_sets)
    if shared_ids:
        print(f"Checked. {len(shared_ids)} IDs are found in all three DataFrames.")
    else:
        print("No IDs are shared across all three DataFrames.")
else:
    print("One or more DataFrames are missing the 'listing_id' column.")

Checked. 25209 IDs are found in all three DataFrames.


In [125]:
# merge the dfs together using listing_id as the intersection, we should expect 7 columns
wide_df = (
    price_df
    .merge(roomtype_df, on='listing_id', how='inner')
    .merge(lastreview_df, on='listing_id', how='inner')
)
wide_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
 3   description  25199 non-null  object
 4   room_type    25209 non-null  object
 5   host_name    25201 non-null  object
 6   last_review  25209 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [126]:
wide_df.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
1,3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
2,5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
4,5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


### What are the dates of the earliest and most recent reviews?
_Store these values as two separate variables with your preferred names._

In [127]:
earliest_review = wide_df['last_review'].min()
earliest_review

'April 01 2019'

In [128]:
mostrecent_review = wide_df['last_review'].max()
mostrecent_review

'May 31 2019'

This is incorrect, looks like it's sorting the `last_review` column alphabetically which tells me I should of made sure the data-types were cleaned and prepped...

In [129]:
wide_df.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
1,3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
2,5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
4,5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


In [130]:
# prices are currently all string - '### dollars' - convert to int32
wide_df['price'] = wide_df['price'].astype(str)
wide_df['price'] = (
    wide_df['price']
    .str.replace(' dollars', '', regex=False)
    .astype('int32')
)

# convert last_review into datetime
wide_df['last_review'] = pd.to_datetime(wide_df['last_review'])

Initially when I checked this, I was confused as to why the str dtypes were still appearing as objects. This is where I learned about pandas _more recent_ use adoption of the 'string' type, which appears to be more a preference in current workflows...

In [131]:
# convert all text/object columns into str
wide_df[['nbhood_full', 'description', 'room_type', 'host_name']] = wide_df[['nbhood_full', 'description', 'room_type', 'host_name']].astype('string')

# make room type values consistent casing
wide_df['room_type'] = wide_df['room_type'].str.lower()

# check my moves...
wide_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   price        25209 non-null  int32         
 2   nbhood_full  25209 non-null  string        
 3   description  25199 non-null  string        
 4   room_type    25209 non-null  string        
 5   host_name    25201 non-null  string        
 6   last_review  25209 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), int64(1), string(4)
memory usage: 1.4 MB


Let's try again with the earliest and most recent dates 

In [132]:
earliest_review = wide_df['last_review'].min()
earliest_review

Timestamp('2019-01-01 00:00:00')

In [133]:
most_recent_review = wide_df['last_review'].max()
most_recent_review

Timestamp('2019-07-09 00:00:00')

### How many of the listings are private rooms?

In [134]:
private_rooms_only = len(wide_df[wide_df['room_type'] == 'private room'])
private_rooms_only

11356

### What is the average listing price? Round to the nearest two decimal places.

In [135]:
avg_price = wide_df['price'].mean().round(2) 
avg_price

141.78

## Combine all the new variables into one DataFrame

In [136]:
review_dates = pd.DataFrame([{
    'first_reviewed': earliest_review,
    'last_reviewed': most_recent_review,
    'nb_private_rooms': private_rooms_only,
    'avg_price': avg_price
}])
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78
